https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [3]:
import pandas as pd

In [5]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

In [6]:
train_df.head()

,path,class_name
0,fastcampus/dataset/cifar/train\0_frog.png,frog
1,fastcampus/dataset/cifar/train\10000_automobil...,automobile
2,fastcampus/dataset/cifar/train\10001_frog.png,frog
3,fastcampus/dataset/cifar/train\10002_frog.png,frog
4,fastcampus/dataset/cifar/train\10003_ship.png,ship


In [7]:
test_df.head()

,path,class_name
0,fastcampus/dataset/cifar/test\0_cat.png,cat
1,fastcampus/dataset/cifar/test\1000_dog.png,dog
2,fastcampus/dataset/cifar/test\1001_airplane.png,airplane
3,fastcampus/dataset/cifar/test\1002_ship.png,ship
4,fastcampus/dataset/cifar/test\1003_deer.png,deer


In [8]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.3,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_dataframe(
        train_df,
        x_col='path',
        y_col='class_name',
        target_size=input_shape[:2],
        batch_size=batch_size
)

validation_generator = test_datagen.flow_from_dataframe(
        test_df,
        x_col='path',
        y_col='class_name',
        target_size=input_shape[:2],
        batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [10]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [12]:
model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=num_epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1563/1563 [==============================] - 228s 146ms/step - loss: 1.9074 - accuracy: 0.2834 - val_loss: 1.5978 - val_accuracy: 0.4196
Epoch 2/10
1563/1563 [==============================] - 188s 120ms/step - loss: 1.6880 - accuracy: 0.3699 - val_loss: 1.5189 - val_accuracy: 0.4480
Epoch 3/10
1563/1563 [==============================] - 182s 117ms/step - loss: 1.6022 - accuracy: 0.4131 - val_loss: 1.5049 - val_accuracy: 0.4500
Epoch 4/10
1563/1563 [==============================] - 177s 113ms/step - loss: 1.5531 - accuracy: 0.4304 - val_loss: 1.5330 - val_accuracy: 0.4121
Epoch 5/10
1563/1563 [==============================] - 245s 157ms/step - loss: 1.5250 - accuracy: 0.4435 - val_loss: 1.4084 - val_accuracy: 0.4594
Epoch 6/10
1563/1563 [==============================] - 197s 126ms/step - loss: 1.4939 - accuracy: 0.4551 - val_loss: 1.5212 - val_accuracy: 0.4366
Epoch 7/10
1563/15

In [14]:
len(train_generator)
## train_generator는 아래 train_df/batch size와 같다.
## 한편 train_df는 전체 인풋의 갯수이므로, train_generator에서는 각 배치사이즈를 한 셋으로 1563개의 세트를 넣어줌


1563

In [15]:
len(train_df)//32
## train_df를 배치사이즈로 나눈거

1562